# DSPy

```terminal
uv pip install -U dspy
```

In [1]:
from textwrap import dedent

import dspy
from IPython.display import Latex, Markdown, display
from rich.pretty import pprint
from gait import s_message, u_message

## Define the LLM

In [2]:
lm = dspy.LM(
    "ollama_chat/phi4:14b-q8_0",
    api_base="http://localhost:11434",
    api_key="ollama",
)
dspy.configure(lm=lm)

In [3]:
lm("What is the meaning of the number 42 in SciFi?")

['In science fiction, particularly within Douglas Adams\' "The Hitchhiker\'s Guide to the Galaxy" series, the number 42 holds a special significance as it is humorously presented as the "Answer to the Ultimate Question of Life, the Universe, and Everything." This answer is provided by an enormous supercomputer named Deep Thought after seven and a half million years of computation.\n\nThe irony and humor in this concept stem from the fact that while the number 42 is given as the definitive answer, the actual question it answers remains unknown. Throughout the series, characters embark on various quests to discover what this ultimate question might be, highlighting themes of absurdity and the search for meaning in a seemingly indifferent universe.\n\nThe use of 42 has since become a cultural reference point and an inside joke among fans of science fiction and those who appreciate its satirical take on existential questions.']

In [4]:
for _ in lm.history:
    pprint(_, expand_all=True)

{
│   'prompt': 'What is the meaning of the number 42 in SciFi?',
│   'messages': None,
│   'kwargs': {},
│   'response': ModelResponse(
│   │   id='chatcmpl-0cf5cb90-bdf1-4983-941b-92782aeb3b6f',
│   │   created=1749687659,
│   │   model='ollama_chat/phi4:14b-q8_0',
│   │   object='chat.completion',
│   │   system_fingerprint=None,
│   │   choices=[
│   │   │   Choices(
│   │   │   │   finish_reason='stop',
│   │   │   │   index=0,
│   │   │   │   message=Message(
│   │   │   │   │   content='In science fiction, particularly within Douglas Adams\' "The Hitchhiker\'s Guide to the Galaxy" series, the number 42 holds a special significance as it is humorously presented as the "Answer to the Ultimate Question of Life, the Universe, and Everything." This answer is provided by an enormous supercomputer named Deep Thought after seven and a half million years of computation.\n\nThe irony and humor in this concept stem from the fact that while the number 42 is given as the definitive answer, the actual question it answers remains unknown. Throughout the series, characters embark on various quests to discover what this ultimate question might be, highlighting themes of absurdity and the search for meaning in a seemingly indifferent universe.\n\nThe use of 42 has since become a cultural reference point and an inside joke among fans of science fiction and those who appreciate its satirical take on existential questions.',
│   │   │   │   │   role='assistant',
│   │   │   │   │   tool_calls=None,
│   │   │   │   │   function_call=None,
│   │   │   │   │   provider_specific_fields=None
│   │   │   │   )
│   │   │   )
│   │   ],
│   │   usage={}
│   ),
│   'outputs': [
│   │   'In science fiction, particularly within Douglas Adams\' "The Hitchhiker\'s Guide to the Galaxy" series, the number 42 holds a special significance as it is humorously presented as the "Answer to the Ultimate Question of Life, the Universe, and Everything." This answer is provided by an enormous supercomputer named Deep Thought after seven and a half million years of computation.\n\nThe irony and humor in this concept stem from the fact that while the number 42 is given as the definitive answer, the actual question it answers remains unknown. Throughout the series, characters embark on various quests to discover what this ultimate question might be, highlighting themes of absurdity and the search for meaning in a seemingly indifferent universe.\n\nThe use of 42 has since become a cultural reference point and an inside joke among fans of science fiction and those who appreciate its satirical take on existential questions.'
│   ],
│   'usage': {},
│   'cost': 0.0,
│   'timestamp': '2025-06-12T06:52:23.124869',
│   'uuid': '11a173d0-0d96-441f-87c1-c669a8bd604a',
│   'model': 'ollama_chat/phi4:14b-q8_0',
│   'response_model': 'ollama_chat/phi4:14b-q8_0',
│   'model_type': 'chat'
}

In [3]:
math = dspy.ChainOfThought("question -> answer: float")
pred = math(
    # question="Two dice are tossed. What is the probability that the sum equals two?"
    question=dedent(
        """
    Think step by step when answering the following >>>question<<<.
    But only keep a minimum draft of each thinking step, with 5 words at most.
    >>>12 - 8 + 9<<<
    """
    ).strip()
)

In [4]:
pprint(pred, expand_all=True)

Prediction(
│   reasoning='Subtract 8 from 12.  \nResult is 4.  \nAdd 9 to result.  \nFinal sum is 13.',
│   answer=13.0
)

In [5]:
print(pred.reasoning)

Subtract 8 from 12.  
Result is 4.  
Add 9 to result.  
Final sum is 13.


In [6]:
display(
    Markdown(
        pred.reasoning
        # Fix Latex
        .replace("\\(", "\\$")
        .replace("\\)", "\\$")
        .replace("\\[", "$$")
        .replace("\\]", "$$")
    )
)

Subtract 8 from 12.  
Result is 4.  
Add 9 to result.  
Final sum is 13.

In [7]:
def evaluate_math(expression: str):
    return dspy.PythonInterpreter({}).execute(expression)

def search_wikipedia(query: str):
    results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=3)
    return [x['text'] for x in results]

react = dspy.ReAct("question -> answer: float", tools=[evaluate_math, search_wikipedia])

pred = react(question="What is 9362158 divided by the year of birth of David Gregory of Kinnairdy castle?")
print(pred.answer)

5761.6


In [16]:
def calculate_percentage(value: float, percentage: float) -> float:
    """Calculate what percentage of a value is.

    :param value: The value to calculate the percentage of.
    :param percentage: The percent value. Has to be betwee 0 and 100.
    :return: The percentage value.
    """
    return value * percentage / 100.0


def subtract(a: float, b: float) -> float:
    """Subtract b from a.

    :param a: The value to subtract from.
    :param b: How much to subtract from a.
    :return: The residual.
    """
    return a - b

In [17]:
react = dspy.ReAct("question -> answer: float", tools=[calculate_percentage, subtract])

pred = react(question="If I have 15 apples and give away 40%, how many are left?")
print(pred.answer)

9.0


In [19]:
for _ in react.history:
    pprint(_, expand_all=True)

{
│   'prompt': None,
│   'messages': [
│   │   {
│   │   │   'role': 'system',
│   │   │   'content': 'Your input fields are:\n1. `question` (str): \n2. `trajectory` (str):\nYour output fields are:\n1. `next_thought` (str): \n2. `next_tool_name` (Literal[\'calculate_percentage\', \'subtract\', \'finish\']): \n3. `next_tool_args` (dict[str, Any]):\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## question ## ]]\n{question}\n\n[[ ## trajectory ## ]]\n{trajectory}\n\n[[ ## next_thought ## ]]\n{next_thought}\n\n[[ ## next_tool_name ## ]]\n{next_tool_name}        # note: the value you produce must exactly match (no extra characters) one of: calculate_percentage; subtract; finish\n\n[[ ## next_tool_args ## ]]\n{next_tool_args}        # note: the value you produce must adhere to the JSON schema: {"type": "object", "additionalProperties": true}\n\n[[ ## completed ## ]]\nIn adhering to this structure, your objective is: \n        Given the fields `question`, produce the fields `answer`.\n        \n        You are an Agent. In each episode, you will be given the fields `question` as input. And you can see your past trajectory so far.\n        Your goal is to use one or more of the supplied tools to collect any necessary information for producing `answer`.\n        \n        To do this, you will interleave next_thought, next_tool_name, and next_tool_args in each turn, and also when finishing the task.\n        After each tool call, you receive a resulting observation, which gets appended to your trajectory.\n        \n        When writing next_thought, you may reason about the current situation and plan for future steps.\n        When selecting the next_tool_name and its next_tool_args, the tool must be one of:\n        \n        (1) calculate_percentage, whose description is <desc>Calculate what percentage of a value is.        :param value: The value to calculate the percentage of.      :param percentage: The percent value. Has to be betwee 0 and 100.      :return: The percentage value.      </desc>. It takes arguments {\'value\': {\'type\': \'number\'}, \'percentage\': {\'type\': \'number\'}}.\n        (2) subtract, whose description is <desc>Subtract b from a.        :param a: The value to subtract from.      :param b: How much to subtract from a.      :return: The residual.      </desc>. It takes arguments {\'a\': {\'type\': \'number\'}, \'b\': {\'type\': \'number\'}}.\n        (3) finish, whose description is <desc>Marks the task as complete. That is, signals that all information for producing the outputs, i.e. `answer`, are now available to be extracted.</desc>. It takes arguments {}.\n        When providing `next_tool_args`, the value inside the field must be in JSON format'
│   │   },
│   │   {
│   │   │   'role': 'user',
│   │   │   'content': "[[ ## question ## ]]\nIf I have 15 apples and give away 40%, how many are left?\n\n[[ ## trajectory ## ]]\n\n\nRespond with the corresponding output fields, starting with the field `[[ ## next_thought ## ]]`, then `[[ ## next_tool_name ## ]]` (must be formatted as a valid Python Literal['calculate_percentage', 'subtract', 'finish']), then `[[ ## next_tool_args ## ]]` (must be formatted as a valid Python dict[str, Any]), and then ending with the marker for `[[ ## completed ## ]]`."
│   │   }
│   ],
│   'kwargs': {},
│   'response': ModelResponse(
│   │   id='chatcmpl-6d1c4095-9b6c-46ef-b675-2bc236467a0c',
│   │   created=1749674572,
│   │   model='ollama_chat/phi4:14b-q8_0',
│   │   object='chat.completion',
│   │   system_fingerprint=None,
│   │   choices=[
│   │   │   Choices(
│   │   │   │   finish_reason='stop',
│   │   │   │   index=0,
│   │   │   │   message=Message(
│   │   │   │   │   content='[[ ## next_thought ## ]]\nTo determine how many apples are left after giving away 40%, I first need to calculate what 40% of 15 apples is. Once I have that number, I can subtract it from the original total of 15 apples.\n\n[[ ## next_

{
│   'prompt': None,
│   'messages': [
│   │   {
│   │   │   'role': 'system',
│   │   │   'content': 'Your input fields are:\n1. `question` (str): \n2. `trajectory` (str):\nYour output fields are:\n1. `next_thought` (str): \n2. `next_tool_name` (Literal[\'calculate_percentage\', \'subtract\', \'finish\']): \n3. `next_tool_args` (dict[str, Any]):\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## question ## ]]\n{question}\n\n[[ ## trajectory ## ]]\n{trajectory}\n\n[[ ## next_thought ## ]]\n{next_thought}\n\n[[ ## next_tool_name ## ]]\n{next_tool_name}        # note: the value you produce must exactly match (no extra characters) one of: calculate_percentage; subtract; finish\n\n[[ ## next_tool_args ## ]]\n{next_tool_args}        # note: the value you produce must adhere to the JSON schema: {"type": "object", "additionalProperties": true}\n\n[[ ## completed ## ]]\nIn adhering to this structure, your objective is: \n        Given the fields `question`, produce the fields `answer`.\n        \n        You are an Agent. In each episode, you will be given the fields `question` as input. And you can see your past trajectory so far.\n        Your goal is to use one or more of the supplied tools to collect any necessary information for producing `answer`.\n        \n        To do this, you will interleave next_thought, next_tool_name, and next_tool_args in each turn, and also when finishing the task.\n        After each tool call, you receive a resulting observation, which gets appended to your trajectory.\n        \n        When writing next_thought, you may reason about the current situation and plan for future steps.\n        When selecting the next_tool_name and its next_tool_args, the tool must be one of:\n        \n        (1) calculate_percentage, whose description is <desc>Calculate what percentage of a value is.        :param value: The value to calculate the percentage of.      :param percentage: The percent value. Has to be betwee 0 and 100.      :return: The percentage value.      </desc>. It takes arguments {\'value\': {\'type\': \'number\'}, \'percentage\': {\'type\': \'number\'}}.\n        (2) subtract, whose description is <desc>Subtract b from a.        :param a: The value to subtract from.      :param b: How much to subtract from a.      :return: The residual.      </desc>. It takes arguments {\'a\': {\'type\': \'number\'}, \'b\': {\'type\': \'number\'}}.\n        (3) finish, whose description is <desc>Marks the task as complete. That is, signals that all information for producing the outputs, i.e. `answer`, are now available to be extracted.</desc>. It takes arguments {}.\n        When providing `next_tool_args`, the value inside the field must be in JSON format'
│   │   },
│   │   {
│   │   │   'role': 'user',
│   │   │   'content': '[[ ## question ## ]]\nIf I have 15 apples and give away 40%, how many are left?\n\n[[ ## trajectory ## ]]\n[[ ## thought_0 ## ]]\nTo determine how many apples are left after giving away 40%, I first need to calculate what 40% of 15 apples is. Once I have that number, I can subtract it from the original total of 15 apples.\n\n[[ ## tool_name_0 ## ]]\ncalculate_percentage\n\n[[ ## tool_args_0 ## ]]\n{"value": 15, "percentage": 40}\n\n[[ ## observation_0 ## ]]\n6.0\n\nRespond with the corresponding output fields, starting with the field `[[ ## next_thought ## ]]`, then `[[ ## next_tool_name ## ]]` (must be formatted as a valid Python Literal[\'calculate_percentage\', \'subtract\', \'finish\']), then `[[ ## next_tool_args ## ]]` (must be formatted as a valid Python dict[str, Any]), and then ending with the marker for `[[ ## completed ## ]]`.'
│   │   }
│   ],
│   'kwargs': {},
│   'response': ModelResponse(
│   │   id='chatcmpl-0fa2a38e-8eb3-4bd5-88c4-f6c6664ec3c4',
│   │   created=1749674577,
│   │   model='ollama_chat/phi4:14b-q8_0',
│   │   object='chat.completion',
│   │   system_fingerprint=None,
│   │   choices=[
│   │   │   Cho

{
│   'prompt': None,
│   'messages': [
│   │   {
│   │   │   'role': 'system',
│   │   │   'content': 'Your input fields are:\n1. `question` (str): \n2. `trajectory` (str):\nYour output fields are:\n1. `next_thought` (str): \n2. `next_tool_name` (Literal[\'calculate_percentage\', \'subtract\', \'finish\']): \n3. `next_tool_args` (dict[str, Any]):\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## question ## ]]\n{question}\n\n[[ ## trajectory ## ]]\n{trajectory}\n\n[[ ## next_thought ## ]]\n{next_thought}\n\n[[ ## next_tool_name ## ]]\n{next_tool_name}        # note: the value you produce must exactly match (no extra characters) one of: calculate_percentage; subtract; finish\n\n[[ ## next_tool_args ## ]]\n{next_tool_args}        # note: the value you produce must adhere to the JSON schema: {"type": "object", "additionalProperties": true}\n\n[[ ## completed ## ]]\nIn adhering to this structure, your objective is: \n        Given the fields `question`, produce the fields `answer`.\n        \n        You are an Agent. In each episode, you will be given the fields `question` as input. And you can see your past trajectory so far.\n        Your goal is to use one or more of the supplied tools to collect any necessary information for producing `answer`.\n        \n        To do this, you will interleave next_thought, next_tool_name, and next_tool_args in each turn, and also when finishing the task.\n        After each tool call, you receive a resulting observation, which gets appended to your trajectory.\n        \n        When writing next_thought, you may reason about the current situation and plan for future steps.\n        When selecting the next_tool_name and its next_tool_args, the tool must be one of:\n        \n        (1) calculate_percentage, whose description is <desc>Calculate what percentage of a value is.        :param value: The value to calculate the percentage of.      :param percentage: The percent value. Has to be betwee 0 and 100.      :return: The percentage value.      </desc>. It takes arguments {\'value\': {\'type\': \'number\'}, \'percentage\': {\'type\': \'number\'}}.\n        (2) subtract, whose description is <desc>Subtract b from a.        :param a: The value to subtract from.      :param b: How much to subtract from a.      :return: The residual.      </desc>. It takes arguments {\'a\': {\'type\': \'number\'}, \'b\': {\'type\': \'number\'}}.\n        (3) finish, whose description is <desc>Marks the task as complete. That is, signals that all information for producing the outputs, i.e. `answer`, are now available to be extracted.</desc>. It takes arguments {}.\n        When providing `next_tool_args`, the value inside the field must be in JSON format'
│   │   },
│   │   {
│   │   │   'role': 'user',
│   │   │   'content': '[[ ## question ## ]]\nIf I have 15 apples and give away 40%, how many are left?\n\n[[ ## trajectory ## ]]\n[[ ## thought_0 ## ]]\nTo determine how many apples are left after giving away 40%, I first need to calculate what 40% of 15 apples is. Once I have that number, I can subtract it from the original total of 15 apples.\n\n[[ ## tool_name_0 ## ]]\ncalculate_percentage\n\n[[ ## tool_args_0 ## ]]\n{"value": 15, "percentage": 40}\n\n[[ ## observation_0 ## ]]\n6.0\n\n[[ ## thought_1 ## ]]\nI have calculated that 40% of 15 apples is 6 apples. Now, I need to subtract these 6 apples from the original total of 15 apples to find out how many are left.\n\n[[ ## tool_name_1 ## ]]\nsubtract\n\n[[ ## tool_args_1 ## ]]\n{"a": 15, "b": 6}\n\n[[ ## observation_1 ## ]]\n9.0\n\nRespond with the corresponding output fields, starting with the field `[[ ## next_thought ## ]]`, then `[[ ## next_tool_name ## ]]` (must be formatted as a valid Python Literal[\'calculate_percentage\', \'subtract\', \'finish\']), then `[[ ## next_tool_args ## ]]` (must be formatted as a valid Python dict[str, Any]), and then ending with the marker for `[[ ## completed ## 

{
│   'prompt': None,
│   'messages': [
│   │   {
│   │   │   'role': 'system',
│   │   │   'content': 'Your input fields are:\n1. `question` (str): \n2. `trajectory` (str):\nYour output fields are:\n1. `reasoning` (str): \n2. `answer` (float):\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## question ## ]]\n{question}\n\n[[ ## trajectory ## ]]\n{trajectory}\n\n[[ ## reasoning ## ]]\n{reasoning}\n\n[[ ## answer ## ]]\n{answer}        # note: the value you produce must be a single float value\n\n[[ ## completed ## ]]\nIn adhering to this structure, your objective is: \n        Given the fields `question`, produce the fields `answer`.'
│   │   },
│   │   {
│   │   │   'role': 'user',
│   │   │   'content': '[[ ## question ## ]]\nIf I have 15 apples and give away 40%, how many are left?\n\n[[ ## trajectory ## ]]\n[[ ## thought_0 ## ]]\nTo determine how many apples are left after giving away 40%, I first need to calculate what 40% of 15 apples is. Once I have that number, I can subtract it from the original total of 15 apples.\n\n[[ ## tool_name_0 ## ]]\ncalculate_percentage\n\n[[ ## tool_args_0 ## ]]\n{"value": 15, "percentage": 40}\n\n[[ ## observation_0 ## ]]\n6.0\n\n[[ ## thought_1 ## ]]\nI have calculated that 40% of 15 apples is 6 apples. Now, I need to subtract these 6 apples from the original total of 15 apples to find out how many are left.\n\n[[ ## tool_name_1 ## ]]\nsubtract\n\n[[ ## tool_args_1 ## ]]\n{"a": 15, "b": 6}\n\n[[ ## observation_1 ## ]]\n9.0\n\n[[ ## thought_2 ## ]]\nI have successfully calculated that 40% of 15 apples is 6 apples. After subtracting these 6 apples from the original total of 15 apples, I found that there are 9 apples left.\n\n[[ ## tool_name_2 ## ]]\nfinish\n\n[[ ## tool_args_2 ## ]]\n{}\n\n[[ ## observation_2 ## ]]\nCompleted.\n\nRespond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## answer ## ]]` (must be formatted as a valid Python float), and then ending with the marker for `[[ ## completed ## ]]`.'
│   │   }
│   ],
│   'kwargs': {},
│   'response': ModelResponse(
│   │   id='chatcmpl-ba08b13d-18b4-463d-84d4-40993faf84e0',
│   │   created=1749674589,
│   │   model='ollama_chat/phi4:14b-q8_0',
│   │   object='chat.completion',
│   │   system_fingerprint=None,
│   │   choices=[
│   │   │   Choices(
│   │   │   │   finish_reason='stop',
│   │   │   │   index=0,
│   │   │   │   message=Message(
│   │   │   │   │   content='[[ ## reasoning ## ]]\nTo determine how many apples are left after giving away 40%, I first calculated what 40% of 15 apples is. This calculation resulted in 6 apples being given away (since 40% of 15 equals 6). Next, I subtracted these 6 apples from the original total of 15 apples to find out how many are left. The subtraction of 6 from 15 results in 9 apples remaining.\n\n[[ ## answer ## ]]\n9.0\n\n[[ ## completed ## ]]',
│   │   │   │   │   role='assistant',
│   │   │   │   │   tool_calls=None,
│   │   │   │   │   function_call=None,
│   │   │   │   │   provider_specific_fields=None
│   │   │   │   )
│   │   │   )
│   │   ],
│   │   usage={},
│   │   cache_hit=None
│   ),
│   'outputs': [
│   │   '[[ ## reasoning ## ]]\nTo determine how many apples are left after giving away 40%, I first calculated what 40% of 15 apples is. This calculation resulted in 6 apples being given away (since 40% of 15 equals 6). Next, I subtracted these 6 apples from the original total of 15 apples to find out how many are left. The subtraction of 6 from 15 results in 9 apples remaining.\n\n[[ ## answer ## ]]\n9.0\n\n[[ ## completed ## ]]'
│   ],
│   'usage': {},
│   'cost': 0.0,
│   'timestamp': '2025-06-11T18:51:12.875393',
│   'uuid': '9afe2c30-aa19-4df2-b541-096337fc945c',
│   'model': 'ollama_chat/phi4:14b-q8_0',
│   'response_model': 'ollama_chat/phi4:14b-q8_0',
│   'model_type': 'chat'
}